In [ ]:
import pandas as pd
import numpy as np

In [2]:
def read_bd(path):
    
    df_universo = pd.read_csv(path+'dbUniverso.csv',delimiter='|', dtype = {'personal_id': str,'customer_id':str,'sbs_customer_id':str,'taxpayer_id':str})
    df_universo = df_universo.drop_duplicates()
    #,dtype='str'
    
    #Información general por pago de impuestos
    df_sunat = pd.read_csv(path+'dbSunat.csv',delimiter='|', dtype = {'personal_id': str,'taxpayer_id':str})
    df_sunat = df_sunat.drop_duplicates()
    
    # Vehiculos y bienes (pueden estar duplicados)
    df_sunart = pd.read_csv(path+'dbSunarp.csv',delimiter='|', dtype = {'personal_id': str})
    df_sunart = df_sunart.drop_duplicates()
    
    # Buró de crédito
    df_rcc = pd.read_csv(path+'dbRcc.csv',delimiter='|', dtype = {'sbs_customer_id':str})
    df_rcc = df_rcc.drop_duplicates()
    
    #Información general sobre indicadores de la empresa
    df_ranking = pd.read_csv(path+'dbRanking.csv',delimiter='|', dtype = {'taxpayer_id':str})
    df_ranking = df_ranking.drop_duplicates()
    
    #Digitalización de clientes
    df_digital = pd.read_csv(path+'dbAfilDigital.csv',delimiter='|',dtype = {'customer_id':str})
    df_digital = df_digital.drop_duplicates()

    #Transacciones 
    df_transac = pd.read_csv(path+'dbTransacPyme_v2.csv',delimiter='|',dtype = {'COD_PERSONOR': str,'COD_PERSONDE':str})
    df_transac = df_transac.drop_duplicates()

    return (df_universo, df_sunat, df_sunart, df_rcc, df_ranking, df_digital, df_transac)


In [1]:


def clean_db_ranking(df):
    ranking_dict = ['ranking_'+str(i)+'_number' for i in range(2017,2009,-1)]
    min_usd_billing_dict = ['min_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
    max_usd_billing_dict = ['max_usd_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
    min_pen_billing_dict = ['min_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)] 
    max_pen_billing_dict = ['max_pen_billing_'+str(i)+'_amount' for i in range(2017,2012,-1)]
    pbi_sector_dict = ['pbi_sector_'+str(i)+'_per' for i in range(2016,2006,-1)]
    
    
    df = df.drop_duplicates(ignore_index=True)
    df = df.dropna(axis=1, how='all')
    df = df[df['market_share_per'] < 99] # Removing, these are like 10 registers
    df = df.drop(['ebitda_2014_amount', 'ebitda_2013_amount'], axis=1) # Removing these columns because are too few
    
    for col in ranking_dict: #computing and droping ranking
        df['ranking_2017_number'].fillna(df[col], inplace=True)
    df['ranking_last_number'] = df['ranking_2017_number']
    df = df.drop(ranking_dict, axis=1)
    
    for col in min_usd_billing_dict: #computing and dropping min usd amounts
        df['min_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_usd_billing_last_amount'] = df['min_usd_billing_2016_amount']
    df = df.drop(min_usd_billing_dict, axis=1)

    for col in max_usd_billing_dict: #computing and dropping max usd amounts
        df['max_usd_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_usd_billing_last_amount'] = df['max_usd_billing_2017_amount']
    df = df.drop(max_usd_billing_dict, axis=1)

    for col in min_pen_billing_dict: #computing and dropping max usd amounts
        df['min_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['min_pen_billing_last_amount'] = df['min_pen_billing_2017_amount']
    df = df.drop(min_pen_billing_dict, axis=1)

    for col in max_pen_billing_dict: #computing and dropping max usd amounts
        df['max_pen_billing_2017_amount'].fillna(df[col], inplace=True)
    df['max_pen_billing_last_amount'] = df['max_pen_billing_2017_amount']
    df = df.drop(max_pen_billing_dict, axis=1)

    df['exporter_mark_type_flag'] = df['exporter_mark_type']
    df['importer_mark_type_flag'] = df['importer_mark_type']

    df = df.drop(df.columns[[i for i in range(15, 185)]], axis=1)

    for col in pbi_sector_dict: #computing and dropping max usd amounts
        df['pbi_sector_2016_per'].fillna(df[col], inplace=True)
    df['pbi_sector_last_per'] = df['pbi_sector_2016_per']
    df['pbi_sector_last_per'] = df['pbi_sector_last_per'].replace(0, np.nan)
    df = df.drop(pbi_sector_dict, axis=1)

    df = df.drop(['cutoff_date'], axis=1)

    return df

In [ ]:
def transf_sunart():
    df_sunart_1 = df_sunart.groupby(['personal_id']).agg(veh_ct=('personal_type','count'), veh_year=('vehicle_manufacture_year_id','max')).reset_index()
    
   
    return df_sunart_1
        

In [ ]:
def transf_rcc():
    
    df_rcc.loc[df_rcc.situacion_credito=='VIGENTE','vo']=1
    df_rcc.loc[df_rcc.situacion_credito=='JUDICIAL','vo']=0
    df_rcc.loc[df_rcc.situacion_credito=='VENCIDO','vo']=0
    df_rcc.loc[df_rcc.situacion_credito=='REFINANCIADO','vo']=0

    df_rcc.loc[df_rcc.situacion_credito=='VIGENTE','vo_balance_amount']=df_rcc.balance_amount
    df_rcc.loc[df_rcc.situacion_credito=='JUDICIAL','vo_balance_amount']=df_rcc.balance_amount*-1
    df_rcc.loc[df_rcc.situacion_credito=='VENCIDO','vo_balance_amount']=df_rcc.balance_amount*-1
    df_rcc.loc[df_rcc.situacion_credito=='REFINANCIADO','vo_balance_amount']=df_rcc.balance_amount*-1

    df_rcc_2 = df_rcc.groupby(['sbs_customer_id','situacion_credito']).agg(vo=('vo','max'), vo_balance_amount=('vo_balance_amount','sum')).reset_index()

    ####
    df_rcc_3 =df_rcc_2.groupby('sbs_customer_id').agg(sm=('situacion_credito','count')).reset_index()
    df_rcc_4 = df_rcc_3[df_rcc_3.sm>0]
    #####

    df_rcc_5 = df_rcc_2.merge(df_rcc_4, how='inner', on='sbs_customer_id')
    
    return df_rcc_5


In [ ]:
def transf_digital(df_digital):
    
    df_digital['u_digital_affiliation_type'] = df_digital['digital_affiliation_type'].map(cat_ind)
    df_digital['u_online_banking_affiliation_type'] = df_digital['online_banking_affiliation_type'].map(cat_ind)
    df_digital['u_sms_affiliation_type'] = df_digital['sms_affiliation_type'].map(cat_ind)

    df_digital['digital_affiliation_date_n']=df_digital['digital_affiliation_date'].replace('9999-12-31','1900-01-01')
    df_digital['digital_affiliation_date_d'] = pd.to_datetime(df_digital['digital_affiliation_date_n'], format='%Y-%m-%d')

    df_digital['online_banking_affiliation_date_n']=df_digital['online_banking_affiliation_date'].replace('9999-12-31','1900-01-01')
    df_digital['online_banking_affiliation_date_d'] = pd.to_datetime(df_digital['online_banking_affiliation_date_n'], format='%Y-%m-%d')

    df_digital['sms_affiliation_date_n']=df_digital['sms_affiliation_date'].replace('9999-12-31','1900-01-01')
    df_digital['sms_affiliation_date_d'] = pd.to_datetime(df_digital['sms_affiliation_date_n'], format='%Y-%m-%d')

    df_digital['cutoff_date_d'] = pd.to_datetime(df_digital['cutoff_date'], format='%Y-%m-%d')

    df_digital['dias_digital'] = (df_digital.cutoff_date_d - df_digital.digital_affiliation_date_d).dt.days
    df_digital['dias_online'] = (df_digital.cutoff_date_d - df_digital.online_banking_affiliation_date_d).dt.days
    df_digital['dias_sms'] = (df_digital.cutoff_date_d - df_digital.sms_affiliation_date_d).dt.days
    
    return df_digital




In [ ]:
def join_tables():
    
    df_join = df_universo.merge(df_sunat, how='left', on='taxpayer_id')\
            .merge(df_sunart_1, how='left', left_on='personal_id_x', right_on='personal_id')\
            .merge(df_rcc_5, how='inner', on='sbs_customer_id')
    
    df_join.update(df_join[['income_monthly_amount','birth_date','veh_ct','veh_year']].fillna(0))
    
    return df_join